Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-recipes-univariate/2_run_experiment.png)

<font color="red" size="5"><strong>!Important!</strong> </br>This notebook is outdated and is not supported by the AutoML Team. Please use the supported version ([link](https://github.com/Azure/azureml-examples/tree/main/sdk/python/jobs/automl-standalone-jobs/automl-forecasting-recipes-univariate)).</font>

# Running AutoML experiments

See the `auto-ml-forecasting-univariate-recipe-experiment-settings` notebook on how to determine settings for seasonal features, target lags and whether the series needs to be differenced or not. To make experimentation user-friendly, the user has to specify several parameters: DIFFERENCE_SERIES, TARGET_LAGS and STL_TYPE. Once these parameters are set, the notebook will generate correct transformations and settings to run experiments, generate forecasts, compute inference set metrics and plot forecast vs actuals. It will also convert the forecast from first differences to levels (original units of measurement) if the DIFFERENCE_SERIES parameter is set to True before calculating inference set metrics.

<br/>

The output generated by this notebook is saved in the `experiment_output`folder.

### Setup

In [ ]:
import os
import logging
import pandas as pd
import numpy as np

import azureml.automl.runtime
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import matplotlib.pyplot as plt
from helper_functions import ts_train_test_split, compute_metrics

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig


# set printing options
np.set_printoptions(precision=4, suppress=True, linewidth=100)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

As part of the setup you have already created a **Workspace**. You will also need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.
> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
ws = Workspace.from_config()
amlcompute_cluster_name = "recipe-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == "AmlCompute":
    found = True
    print("Found existing compute target.")
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print("Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=6
    )

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(
        ws, amlcompute_cluster_name, provisioning_config
    )

print("Checking cluster status...")
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(
    show_output=True, min_node_count=None, timeout_in_minutes=20
)

### Data

Here, we will load the data from the csv file and drop the Covid period.

In [ ]:
main_data_loc = "data"
train_file_name = "S4248SM144SCEN.csv"

TARGET_COLNAME = "S4248SM144SCEN"
TIME_COLNAME = "observation_date"
COVID_PERIOD_START = (
    "2020-03-01"  # start of the covid period. To be excluded from evaluation.
)

# load data
df = pd.read_csv(os.path.join(main_data_loc, train_file_name))
df[TIME_COLNAME] = pd.to_datetime(df[TIME_COLNAME], format="%Y-%m-%d")
df.sort_values(by=TIME_COLNAME, inplace=True)

# remove the Covid period
df = df.query('{} <= "{}"'.format(TIME_COLNAME, COVID_PERIOD_START))

### Set parameters

The first set of parameters is based on the analysis performed in the `auto-ml-forecasting-univariate-recipe-experiment-settings` notebook. 

In [ ]:
# set parameters based on the settings notebook analysis
DIFFERENCE_SERIES = True
TARGET_LAGS = None
STL_TYPE = None

Next, define additional parameters to be used in the <a href="https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig?view=azure-ml-py"> AutoML config </a> class.

<ul> 
    <li> FORECAST_HORIZON:  The forecast horizon is the number of periods into the future that the model should predict. Here, we set the horizon to 12 periods (i.e. 12 quarters). For more discussion of forecast horizons and guiding principles for setting them, please see the <a href="https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand"> energy demand notebook </a>. 
    </li>
    <li> TIME_SERIES_ID_COLNAMES: The names of columns used to group a timeseries. It can be used to create multiple series. If time series identifier is not defined, the data set is assumed to be one time-series. This parameter is used with task type forecasting. Since we are working with a single series, this list is empty.
    </li>
    <li> BLOCKED_MODELS: Optional list of models to be blocked from consideration during model selection stage. At this point we want to consider all ML and Time Series models.
        <ul>
            <li> See the following <a href="https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.forecasting?view=azure-ml-py"> link </a> for a list of supported Forecasting models</li>
        </ul>
    </li>
</ul>


In [ ]:
# set other parameters
FORECAST_HORIZON = 12
TIME_SERIES_ID_COLNAMES = []
BLOCKED_MODELS = []

To run AutoML, you also need to create an **Experiment**. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
# choose a name for the run history container in the workspace
if isinstance(TARGET_LAGS, list):
    TARGET_LAGS_STR = (
        "-".join(map(str, TARGET_LAGS)) if (len(TARGET_LAGS) > 0) else None
    )
else:
    TARGET_LAGS_STR = TARGET_LAGS

experiment_desc = "diff-{}_lags-{}_STL-{}".format(
    DIFFERENCE_SERIES, TARGET_LAGS_STR, STL_TYPE
)
experiment_name = "alcohol_{}".format(experiment_desc)
experiment = Experiment(ws, experiment_name)

output = {}
output["SDK version"] = azureml.core.VERSION
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["SKU"] = ws.sku
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
print(outputDf.T)

In [ ]:
# create output directory
output_dir = "experiment_output/{}".format(experiment_desc)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# difference data and test for unit root
if DIFFERENCE_SERIES:
    df_delta = df.copy()
    df_delta[TARGET_COLNAME] = df[TARGET_COLNAME].diff()
    df_delta.dropna(axis=0, inplace=True)

In [ ]:
# split the data into train and test set
if DIFFERENCE_SERIES:
    # generate train/inference sets using data in first differences
    df_train, df_test = ts_train_test_split(
        df_input=df_delta,
        n=FORECAST_HORIZON,
        time_colname=TIME_COLNAME,
        ts_id_colnames=TIME_SERIES_ID_COLNAMES,
    )
else:
    df_train, df_test = ts_train_test_split(
        df_input=df,
        n=FORECAST_HORIZON,
        time_colname=TIME_COLNAME,
        ts_id_colnames=TIME_SERIES_ID_COLNAMES,
    )

### Upload files to the Datastore
The [Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-workspace) is paired with the storage account, which contains the default data store. We will use it to upload the bike share data and create [tabular dataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py) for training. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

In [ ]:
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

from azureml.data.dataset_factory import TabularDatasetFactory

datastore = ws.get_default_datastore()
train_dataset = TabularDatasetFactory.register_pandas_dataframe(
    df_train, target=(datastore, "dataset/"), name="train"
)
test_dataset = TabularDatasetFactory.register_pandas_dataframe(
    df_test, target=(datastore, "dataset/"), name="test"
)

# print the first 5 rows of the Dataset
train_dataset.to_pandas_dataframe().reset_index(drop=True).head(5)

### Config AutoML

In [ ]:
time_series_settings = {
    "time_column_name": TIME_COLNAME,
    "forecast_horizon": FORECAST_HORIZON,
    "target_lags": TARGET_LAGS,
    "use_stl": STL_TYPE,
    "blocked_models": BLOCKED_MODELS,
    "time_series_id_column_names": TIME_SERIES_ID_COLNAMES,
}

automl_config = AutoMLConfig(
    task="forecasting",
    debug_log="sample_experiment.log",
    primary_metric="normalized_root_mean_squared_error",
    experiment_timeout_minutes=20,
    iteration_timeout_minutes=5,
    enable_early_stopping=True,
    training_data=train_dataset,
    label_column_name=TARGET_COLNAME,
    n_cross_validations="auto",  # Feel free to set to a small integer (>=2) if runtime is an issue.
    cv_step_size="auto",
    verbosity=logging.INFO,
    max_cores_per_iteration=-1,
    compute_target=compute_target,
    **time_series_settings,
)

We will now run the experiment, you can go to Azure ML portal to view the run details.

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)
remote_run.wait_for_completion()

### Retrieve the Best Run details
Below we retrieve the best Run object from among all the runs in the experiment.

In [ ]:
best_run = remote_run.get_best_child()
best_run

### Inference

We now use the best fitted model from the AutoML Run to make forecasts for the test set. We will do batch scoring on the test dataset which should have the same schema as training dataset.

The inference will run on a remote compute. In this example, it will re-use the training compute.

In [ ]:
test_experiment = Experiment(ws, experiment_name + "_inference")

## Retreiving forecasts from the model
We have created a function called `run_forecast` that submits the test data to the best model determined during the training run and retrieves forecasts. This function uses a helper script `forecasting_script` which is uploaded and expecuted on the remote compute.

In [ ]:
from run_forecast import run_remote_inference

remote_run = run_remote_inference(
    test_experiment=test_experiment,
    compute_target=compute_target,
    train_run=best_run,
    test_dataset=test_dataset,
    target_column_name=TARGET_COLNAME,
)
remote_run.wait_for_completion(show_output=False)

remote_run.download_file("outputs/predictions.csv", f"{output_dir}/predictions.csv")

### Download the prediction result for metrics calcuation
The test data with predictions are saved in artifact `outputs/predictions.csv`. We will use it to calculate accuracy metrics and vizualize predictions versus actuals.

In [ ]:
X_trans = pd.read_csv(f"{output_dir}/predictions.csv", parse_dates=[TIME_COLNAME])
X_trans.head()

In [ ]:
# convert forecast in differences to levels
def convert_fcst_diff_to_levels(fcst, yt, df_orig):
    """Convert forecast from first differences to levels."""
    fcst = fcst.reset_index(drop=False, inplace=False)
    fcst["predicted_level"] = fcst["predicted"].cumsum()
    fcst["predicted_level"] = fcst["predicted_level"].astype(float) + float(yt)
    # merge actuals
    out = pd.merge(
        fcst, df_orig[[TIME_COLNAME, TARGET_COLNAME]], on=[TIME_COLNAME], how="inner"
    )
    out.rename(columns={TARGET_COLNAME: "actual_level"}, inplace=True)
    return out

In [ ]:
if DIFFERENCE_SERIES:
    # convert forecast in differences to the levels
    INFORMATION_SET_DATE = max(df_train[TIME_COLNAME])
    YT = df.query("{} == @INFORMATION_SET_DATE".format(TIME_COLNAME))[TARGET_COLNAME]

    fcst_df = convert_fcst_diff_to_levels(fcst=X_trans, yt=YT, df_orig=df)
else:
    fcst_df = X_trans.copy()
    fcst_df["actual_level"] = y_test
    fcst_df["predicted_level"] = y_predictions

del X_trans

### Calculate metrics and save output

In [ ]:
# compute metrics
metrics_df = compute_metrics(fcst_df=fcst_df, metric_name=None, ts_id_colnames=None)
# save output
metrics_file_name = "{}_metrics.csv".format(experiment_name)
fcst_file_name = "{}_forecst.csv".format(experiment_name)
plot_file_name = "{}_plot.pdf".format(experiment_name)

metrics_df.to_csv(os.path.join(output_dir, metrics_file_name), index=True)
fcst_df.to_csv(os.path.join(output_dir, fcst_file_name), index=True)

### Generate and save visuals

In [ ]:
plot_df = df.query('{} > "2010-01-01"'.format(TIME_COLNAME))
plot_df.set_index(TIME_COLNAME, inplace=True)
fcst_df.set_index(TIME_COLNAME, inplace=True)

# generate and save plots
fig, ax = plt.subplots(dpi=180)
ax.plot(plot_df[TARGET_COLNAME], "-g", label="Historical")
ax.plot(fcst_df["actual_level"], "-b", label="Actual")
ax.plot(fcst_df["predicted_level"], "-r", label="Forecast")
ax.legend()
ax.set_title("Forecast vs Actuals")
ax.set_xlabel(TIME_COLNAME)
ax.set_ylabel(TARGET_COLNAME)
locs, labels = plt.xticks()

plt.setp(labels, rotation=45)
plt.savefig(os.path.join(output_dir, plot_file_name))